In [1]:
import pandas as pd
import re

In [2]:
df =  pd.read_csv('atriographics_030225.csv', sep = ";", encoding="utf-8")

In [3]:
lista = df["Descripción"].str.split().str[0]

In [4]:
sorted(lista.unique())

['(Sesiones',
 'Aceptar',
 'Añadida',
 'Borrado',
 'Cambio',
 'Cargado',
 'Creado',
 'Duración',
 'Error',
 'Generar',
 'Intento',
 'Limpiar',
 'Login',
 'Logout',
 'Nuevo',
 'Solicitud',
 'Visualizar']

In [5]:
len(lista.unique())

17

In [6]:
mapeo_eventos = {
    '(Sesiones': lambda x: "CONEXION" if re.search(r'\bconectada\b', x.lower()) else "DESCONEXION",
    'Aceptar': "ACEPTAR_PRIVACIDAD",       
    'Añadida': "AÑADIR_PIEZA",
    'Borrado': "BORRAR_DISEÑO",          
    'Cambio': "CAMBIO_CONTRASEÑA",              
    'Cargado': "CARGAR_DISENO",
    'Creado': "CREAR_DISENO",
    'Duración': "DURACION",             
    'Error': "ERROR_SISTEMA",          
    'Generar': lambda x: "GEN_PRESUPUESTO" if re.search(r'\bpresupuesto\b', x.lower()) else "GEN_PEDIDO",
    'Intento': "INTENTO_RESET",       
    'Limpiar': "LIMPIAR_DATOS",        
    'Login': "LOGIN",
    'Logout': "LOGOUT",
    'Nuevo': "NUEVO_LOGO",              
    'Solicitud': "SOLICITUD_RESET",            
    'Visualizar': lambda x: "VER_PRESUPUESTO" if re.search(r'\bpresupuesto\b', x.lower()) else "VER_PEDIDO",
}

In [7]:
def categorizar_evento(descripcion):
    descripcion = str(descripcion)
    primera_palabra = descripcion.split()[0]
    
    if primera_palabra in mapeo_eventos:
        if callable(mapeo_eventos[primera_palabra]):
            return mapeo_eventos[primera_palabra](descripcion)
        else:
            return mapeo_eventos[primera_palabra]
    else:
        return "OTRO"

In [8]:
df["Evento"] = df["Descripción"].apply(categorizar_evento)

In [9]:
df[df['Evento'] == 'SOLICITUD_RESET'][['Descripción', 'Evento']].head(10)

,Descripción,Evento
2041,Solicitud resetear contraseña userID: 8724,SOLICITUD_RESET
2140,Solicitud resetear contraseña userID: 17896,SOLICITUD_RESET
6143,Solicitud resetear contraseña userID: 12832,SOLICITUD_RESET
6148,Solicitud resetear contraseña userID: 12832,SOLICITUD_RESET
9567,Solicitud resetear contraseña userID: 20704,SOLICITUD_RESET
9828,Solicitud resetear contraseña userID: 16644,SOLICITUD_RESET
12109,Solicitud resetear contraseña userID: 7910,SOLICITUD_RESET
18238,Solicitud resetear contraseña userID: 1842,SOLICITUD_RESET
22076,Solicitud resetear contraseña userID: 15182,SOLICITUD_RESET
22464,Solicitud resetear contraseña userID: 250,SOLICITUD_RESET


In [10]:
def extraer_columnas(row):
    evento = row['Evento']
    desc = row['Descripción']
    if pd.isna(desc):
        return {}

    if evento == 'AÑADIR_PIEZA':
        return {
            'pieza': buscar(r'pieza ([\w-]+)', desc),
            'piezaid': buscar(r'id (\d+)', desc),
            'catalogo': buscar(r'catálogo (.*?) en el diseño', desc),
            'diseñoid': buscar(r'diseño con id: (-?\d+)', desc)
        }

    elif evento == 'LOGIN' or evento == 'LOGOUT' or evento == 'SOLICITUD_RESET' or evento == 'ACEPTAR_PRIVACIDAD' or evento == 'CAMBIO_CONTRASEÑA':
        return {
            'userid': buscar(r'userID: (\w+)', desc)
        }
    elif evento == 'GEN_PEDIDO':
        return {
            'diseñoid': buscar(r'diseño con id: (-?\d+)', desc),
            'fabricante': buscar(r'fabricante (.*)', desc)
        }

    elif evento == 'CREAR_DISENO' or evento == 'CARGAR_DISENO' or evento == 'GEN_PRESUPUESTO' or evento == 'VER_PRESUPUESTO' or evento == 'ERROR_SISTEMA' or evento == 'BORRAR_DISEÑO' or evento == 'VER_PEDIDO':
        return {
            'diseñoid': buscar(r'diseño con id: (-?\d+)', desc)
        }

    elif evento == 'INTENTO_RESET':
        return {
            'email': buscar(r'email[:\s]+(\S+)', desc)
        }

    elif evento == 'DURACION':
        return {
            'minutos': buscar(r'Duración de la sesión:\s*([\d.]+)', desc)
        }

    elif evento == 'NUEVO_LOGO':
        return {
            'logo': buscar(r'logo[:\s]+(\w+)', desc)
        }

    elif evento == 'CONEXION' or evento == 'DESCONEXION':
        return {
            'nsesiones': buscar(r'activas (\d+)', desc)
        }
    return {}
def buscar(patron, texto):
    match = re.search(patron, texto)
    return match.group(1) if match else None

In [11]:
df_extra = df.apply(extraer_columnas, axis=1, result_type='expand')

In [12]:
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y %H:%M', errors='coerce')

In [13]:
df = pd.concat([df, df_extra], axis=1)

In [14]:
tipo_numerico = ['minutos']
df[tipo_numerico] = df[tipo_numerico].apply(pd.to_numeric, errors='coerce')

In [15]:
tipo_int = ['nsesiones', 'piezaid', 'diseñoid', 'userid']
df[tipo_int] = df[tipo_int].apply(lambda x: x.astype('Int64'))

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74307 entries, 0 to 74306
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Fecha        74307 non-null  datetime64[ns]
 1   Usuario      74307 non-null  object        
 2   ID           74307 non-null  int64         
 3   Acceso       74307 non-null  object        
 4   Descripción  74307 non-null  object        
 5   IP           74307 non-null  object        
 6   Evento       74307 non-null  object        
 7   nsesiones    15323 non-null  Int64         
 8   pieza        32807 non-null  object        
 9   piezaid      32807 non-null  Int64         
 10  catalogo     32807 non-null  object        
 11  diseñoid     42825 non-null  Int64         
 12  userid       10979 non-null  Int64         
 13  minutos      4893 non-null   float64       
 14  fabricante   86 non-null     object        
 15  logo         7 non-null      object        
 16  emai

In [17]:
df.to_parquet("logs.parquet",index=False)

In [18]:
df.to_csv("logs.csv",index=False)